<a href="https://colab.research.google.com/github/chuviethongfpt/atom-assignments/blob/main/HOME_ASSIGNMENT_3_SLACK_API_TO_GSHEET_CVH_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mục đích của bài Assignment**


*   Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
*   **[Optional 1]** Đưa danh sách Users lên Google Spreadsheet, để theo dõi
*   **[Optional 2]** Lấy thông tin Assignment Submission và số Reviews trên #atom-assignmentnt2 và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng** 

*   Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
*   Trích xuất thông tin từ JSON
*   Dùng module gspread để đưa thông tin lên Google Spreadsheet

# 0.Load Modules

In [15]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

# 1.Slack API: User List¶




In [16]:
!ls

 cobalt-catalyst-314307-6b793777d71b.json   sample_data
 env_variable.json			   'Token generate.json'


In [17]:
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    # print(json_data)

In [18]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN']

In [19]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

# TODO #1

In [20]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

In [21]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng
user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [22]:
user_df[user_df.display_name == 'Việt Hồng Chu'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
34,U01UMC0N5U2,chuviethong.vn,Việt Hồng Chu,Việt Hồng Chu,Business Analyst,0899306779,False


# Optional 1: Update data => Google Spreadsheet

# TODO#2

In [23]:
!ls

 cobalt-catalyst-314307-6b793777d71b.json   sample_data
 env_variable.json			   'Token generate.json'


In [25]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'cobalt-catalyst-314307-6b793777d71b.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [26]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1XFv2QcX0u8VpxdR-0OeMeWHQmxwF3vzgrOem67AgASE' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

# Option 2: Ai đã nộp bài?
Slack API: Channel List

In [27]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 
channel_ls = response['channels']

In [28]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'created': 1600856703,
 'creator': 'U01BE2PR6LU',
 'id': 'C01B4PVGLVB',
 'is_archived': False,
 'is_channel': True,
 'is_ext_shared': False,
 'is_general': True,
 'is_group': False,
 'is_im': False,
 'is_member': False,
 'is_mpim': False,
 'is_org_shared': False,
 'is_pending_ext_shared': False,
 'is_private': False,
 'is_shared': False,
 'name': 'general',
 'name_normalized': 'general',
 'num_members': 63,
 'parent_conversation': None,
 'pending_connected_team_ids': [],
 'pending_shared': [],
 'previous_names': [],
 'purpose': {'creator': 'U01BE2PR6LU',
  'last_set': 1600856703,
  'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.'},
 'shared_team_ids': ['T01B7SGGMLJ'],
 'topic': {'creator': '', 'last_set': 0, 'value': ''},
 'unlinked': 0}

# TODO#3
Tìm id của channel #atom-assignment2

In [29]:
for i in range(len(channel_ls)): #Tìm trong các phần tử của channel_ls
    if channel_ls[i]['name'] == 'atom-assignment2': #phần tử nào có name là atom-assignment2
        print(channel_ls[i]['id'])

C021FSDN7LJ


In [30]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C01U6P7LZ8F"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [31]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [32]:
msg_ls[21]

{'attachments': [{'app_unfurl_url': 'https://github.com/saturn1101/datacracy-atom-phuong-tran',
   'bot_id': 'B01UQU8E2A3',
   'color': '24292f',
   'fallback': 'saturn1101/datacracy-atom-phuong-tran',
   'fields': [{'short': True,
     'title': 'Language',
     'value': 'Jupyter Notebook'},
    {'short': True, 'title': 'Last updated', 'value': '6 minutes ago'}],
   'id': 1,
   'is_app_unfurl': True,
   'title': 'saturn1101/datacracy-atom-phuong-tran'},
  {'fallback': 'Google Colaboratory',
   'from_url': 'https://colab.research.google.com/drive/1ilny8bAtOMOT57mVxJDoEZyQWJ_9-UvX?usp=sharing',
   'id': 1,
   'image_bytes': 4534,
   'image_height': 250,
   'image_url': 'https://colab.research.google.com/img/colab_favicon_256px.png',
   'image_width': 250,
   'original_url': 'https://colab.research.google.com/drive/1ilny8bAtOMOT57mVxJDoEZyQWJ_9-UvX?usp=sharing',
   'service_icon': 'https://colab.research.google.com/img/favicon.ico?vrz=colab-20210429-060047-RC00_371090719',
   'service_nam

In [33]:
not_learners_id = ['U01BE2PR6LU']

In [34]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
github, reply_count, reply_users_count, reply_users, latest_reply = '','','','',''
for i in range(20):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: github = github_link[0]
      if 'reply_count' in msg_ls[i].keys(): reply_count = msg_ls[i]['reply_count'] #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): reply_users_count = msg_ls[i]['reply_users_count']
      if 'reply_users' in msg_ls[i].keys(): reply_users = msg_ls[i]['reply_users']
      if 'latest_reply' in msg_ls[i].keys(): latest_reply = dt.fromtimestamp(float(msg_ls[i]['latest_reply']))
      print(ts, user, reply_users_count, reply_users, latest_reply, github)

2021-05-10 04:51:46.004700 U01UJ9LG5U5 2 ['U01USGKQ771', 'U01V00KHHHP'] 2021-05-15 09:14:31.003600 https://github.com/danhpcv/
2021-05-10 04:35:39.003800 U01UMC08KL2 2 ['U01UTGS5ZNY', 'U01UMC08KL2'] 2021-05-18 08:09:59.000800 https://github.com/auslynnguyen/
2021-05-08 06:31:59.005200 U01UMJ36QD9 1 ['U01UMBX2X18'] 2021-05-08 06:36:43.006000 https://github.com/auslynnguyen/
2021-05-04 13:55:46.001800 U01UTGVPE7N 1 ['U01UMBX2X18'] 2021-05-08 06:36:43.006000 https://github.com/hoaintp/


# TODO#4
Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

In [35]:
assignment_dict = {'time': [], 'user': [], 'reply_count': [], 'reply_users_count': [], 'reply_user': [], 'latest_reply': [], 'github': []}
not_learners_id = ['U01BE2PR6LU']
for i in range(40):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  assignment_dict['time'].append(ts)
  assignment_dict['user'].append(msg_ls[i]['user']) # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      #print(msg_ls[i].keys())
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      #print(msg_ls[i])
      if len(github_link) > 0: assignment_dict['github'].append(github_link[0])
      if 'reply_count' in msg_ls[i].keys(): assignment_dict['reply_count'].append(msg_ls[i]['reply_count']) #-> Extract số review
      if 'reply_users_count' in msg_ls[i].keys(): assignment_dict['reply_users_count'].append(msg_ls[i]['reply_users_count'])
      if 'reply_users' in msg_ls[i].keys(): assignment_dict['reply_user'].append(msg_ls[i]['reply_users'])
      if 'latest_reply' in msg_ls[i].keys(): assignment_dict['latest_reply'].append(dt.fromtimestamp(float(msg_ls[i]['latest_reply'])))
assignment_dict


{'github': ['https://github.com/danhpcv/',
  'https://github.com/auslynnguyen/',
  'https://github.com/hoaintp/',
  'https://github.com/Tenderriver/',
  'https://github.com/saturn1101/',
  'https://github.com/AnhThuNguyenHuynh/',
  'https://github.com/vietqh/',
  'https://github.com/vuthanhdatt/',
  'https://github.com/TranPham-96/',
  'https://github.com/annaho124/',
  'https://github.com/minhnguyentk95/',
  'https://github.com/lienanh264/',
  'https://github.com/Lulichlam/',
  'https://github.com/tamle1012/'],
 'latest_reply': [datetime.datetime(2021, 5, 15, 9, 14, 31, 3600),
  datetime.datetime(2021, 5, 18, 8, 9, 59, 800),
  datetime.datetime(2021, 5, 8, 6, 36, 43, 6000),
  datetime.datetime(2021, 5, 11, 3, 10, 6, 1300),
  datetime.datetime(2021, 5, 14, 17, 51, 26, 24600),
  datetime.datetime(2021, 5, 14, 17, 52, 35, 25100),
  datetime.datetime(2021, 5, 11, 12, 37, 14, 3100),
  datetime.datetime(2021, 5, 14, 11, 22, 6, 23600),
  datetime.datetime(2021, 5, 11, 12, 39, 13, 3700),
  da

In [36]:
assignment_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in assignment_dict.items() ]))
assignment_df.head()

,time,user,reply_count,reply_users_count,reply_user,latest_reply,github
0,2021-05-21 04:57:44.000900,U0226MQ73MJ,2.0,2.0,"[U01USGKQ771, U01V00KHHHP]",2021-05-15 09:14:31.003600,https://github.com/danhpcv/
1,2021-05-20 16:43:26.000600,U0226MQ73MJ,2.0,2.0,"[U01UTGS5ZNY, U01UMC08KL2]",2021-05-18 08:09:59.000800,https://github.com/auslynnguyen/
2,2021-05-20 01:58:02.000300,U0226MQ73MJ,1.0,1.0,[U01UMBX2X18],2021-05-08 06:36:43.006000,https://github.com/hoaintp/
3,2021-05-18 16:45:09.001300,U0226MQ73MJ,2.0,2.0,"[U01UMC0N5U2, U01U6JQB695]",2021-05-11 03:10:06.001300,https://github.com/Tenderriver/
4,2021-05-17 17:11:11.002600,U0226MQ73MJ,1.0,1.0,[U01UMBZHU2W],2021-05-14 17:51:26.024600,https://github.com/saturn1101/


In [37]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'cobalt-catalyst-314307-6b793777d71b.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [38]:
sheet_index_no = 0
spreadsheet_key = '1XFv2QcX0u8VpxdR-0OeMeWHQmxwF3vzgrOem67AgASE' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

In [39]:
# ACCESS GOOGLE SHEET
sheet_index_no = 1
spreadsheet_key = '1XFv2QcX0u8VpxdR-0OeMeWHQmxwF3vzgrOem67AgASE' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.add_worksheet(title= 'TODO4', rows= "100", cols= "20") #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, assignment_df) #-> Upload assignment_df vào Sheet 2 trong Spreadsheet
print('Done!')

Done!
